In [1]:
cd ../../src

/Users/in-divye.singh/Documents/Projects/MIC_predictor/src


In [2]:
import biovec
import numpy as np
import pandas as pd
from itertools import chain, combinations
from collections import Counter

from utils import *

from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split

from Bio.SeqUtils.ProtParam import ProteinAnalysis

In [3]:
import seaborn as sns

In [4]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

mape_scorer = make_scorer(mean_absolute_percentage_error, greater_is_better=False)

In [5]:
avp_ic50 = pd.read_csv("../data/raw/AVP-IC50Pred_train.csv")
ha_avp = pd.read_csv("../data/raw/HA_AVP.csv")

In [21]:
df = pd.concat([avp_ic50[['Sequence','MIC']], ha_avp], axis=0).drop_duplicates(['Sequence']).reset_index(drop=True)
df = sequence_filtering(df)

In [22]:
df['pIC50'] = df['MIC'].apply(lambda x: -np.log(x*1e-6))

In [23]:
family = pd.read_csv("../data/raw/712pep_family.csv")

In [24]:
family

,Sequence,Abbreviation,Family
0,KWKVFKKIEKMGRNIRNGIVKAGPAIAVLGEAKAL,JV,Arenaviridae
1,KVLTTGLPALISWIKRKRQQ,JV,Arenaviridae
2,HRILARIRQMMT,PRRSV,Arteriviridae
3,LMRIRQMMT,PRRSV,Arteriviridae
4,HRILMRARQMMT,PRRSV,Arteriviridae
...,...,...,...
702,YTSLIHSLIEEGQNQQEKNEQELLELDKWASLWNWF,HIV,Retroviridae
703,YTSLIQSLIEESQNQQEKNEQQLLELDKWASLWNWF,HIV,Retroviridae
704,KKKKFVKKVAKKVKKVAKKVAKVAVAV,FIV,Retroviridae
705,KRKRFAKKFLRFLRKVIRFLKRFIRRF,FIV,Retroviridae


In [25]:
hiv_cov_seq = family[(family['Abbreviation'] == "HIV")|(family['Abbreviation'] == "SARS-CoV")]

In [26]:
df_hiv_cov = df.merge(hiv_cov_seq,how='right',on='Sequence').reset_index(drop=True)

In [27]:
df_hiv_cov

,Sequence,MIC,pIC50,Abbreviation,Family
0,ACWAAGIKQEF,333.000,8.007368,HIV,Retroviridae
1,ACWGAGIKQEF,333.000,8.007368,HIV,Retroviridae
2,ACWWAGIKAEF,333.000,8.007368,HIV,Retroviridae
3,ACWWAGIKQAF,333.000,8.007368,HIV,Retroviridae
4,ACWWAGIRQEF,333.000,8.007368,HIV,Retroviridae
...,...,...,...,...,...
287,YTSLIHSLIEEGQNQQEKNEQELLELDKWASLWNWF,0.001,20.723266,HIV,Retroviridae
288,YTSLIQSLIEESQNQQEKNEQQLLELDKWASLWNWF,0.001,20.723266,HIV,Retroviridae
289,QKQIANQFNKAISQIQESLTTTSTALGKLQDVVNQNAQALNTLVKQ,3.970,12.436744,SARS-CoV,Coronaviridae
290,QNQSANQFQKEISQINEVLTTTNTSLGKLQDDVNQNNQSLNTLQKE,5.070,12.192170,SARS-CoV,Coronaviridae


In [28]:
def get_physicochemical_properties(df):
    params = ['aromaticity', 'helix', 'turn', 'sheet', 'gravy', 'net_charge_at_pH7point4']

    prop = []
    for seq in df.Sequence:
        X = ProteinAnalysis(seq)
        aromaticity = X.aromaticity()
        sec_struc = X.secondary_structure_fraction()
        helix = sec_struc[0]
        turn = sec_struc[1]
        sheet = sec_struc[2]
        gravy = X.gravy() # hydrophobicity related
        net_charge_at_pH7point4 = X.charge_at_pH(7.4)

        prop.append([aromaticity, helix, turn, sheet, gravy, net_charge_at_pH7point4])
    return pd.DataFrame(prop, columns=params)

In [31]:
aa_freq = reduce_by_kmer_frequency(df_hiv_cov)

In [32]:
uniprot_embedding = biovec.models.load_protvec("../data/embeddings/uniprot__kmer_3_contextWindow_10_vector_100_reduction_None")

avg_protvec = convert_sequences_to_avg_vectors(df_hiv_cov['Sequence'], uniprot_embedding, kmer=3)
avg_protvec = avg_protvec.reset_index(drop=True)

/Users/in-divye.singh/anaconda3/envs/life-science/lib/python3.6/site-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
Creating vectors: 100%|██████████| 292/292 [00:00<00:00, 1077.56sequence/s]


In [33]:
physicochemical_prop = get_physicochemical_properties(df_hiv_cov)

In [34]:
X = pd.concat([aa_freq, avg_protvec, physicochemical_prop[['helix','turn','sheet']]], axis=1)

In [35]:
y = df_hiv_cov[['pIC50', 'MIC']]

In [36]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [37]:
y_train_pmic, y_train_mic = y_train['pIC50'], y_train['MIC']
y_test_pmic, y_test_mic = y_test['pIC50'], y_test['MIC']

In [39]:
from sklearn.model_selection import cross_val_score, GridSearchCV, LeaveOneOut

In [46]:
def multi_objective_score(y_true, y_pred):
    mape = mean_absolute_percentage_error(y_true, y_pred)
    std_diff = abs(np.std(y_pred) - np.std(y_true))
    return mape + 10*std_diff
multi_objective_scorer = make_scorer(multi_objective_score, greater_is_better=False)

In [47]:
param_grid = {
    'C':[0.001,0.01,0.1,1,10,100,1000],
    'kernel':['rbf','poly','sigmoid','linear'],
    'degree':[1,2,3,4,5,6],
    'gamma': np.arange(0.1,1.0,0.1).round(1).tolist() + np.arange(1,11,1).round().tolist()
}
svr = SVR()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = svr, param_grid = param_grid, 
                          cv = 5, n_jobs = -1, verbose = 2, scoring=multi_objective_scorer)

In [48]:
grid_search.fit(X_train, y_train_pmic)

Fitting 5 folds for each of 3192 candidates, totalling 15960 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done 288 tasks      | elapsed:    2.7s
[Parallel(n_jobs=-1)]: Done 3128 tasks      | elapsed:    7.4s
[Parallel(n_jobs=-1)]: Done 7656 tasks      | elapsed:   16.8s
[Parallel(n_jobs=-1)]: Done 13496 tasks      | elapsed:   34.2s
[Parallel(n_jobs=-1)]: Done 15020 tasks      | elapsed:   48.7s
[Parallel(n_jobs=-1)]: Done 15937 out of 15960 | elapsed:   54.7s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done 15960 out of 15960 | elapsed:   54.9s finished


GridSearchCV(cv=5, estimator=SVR(), n_jobs=-1,
             param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
                         'degree': [1, 2, 3, 4, 5, 6],
                         'gamma': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9,
                                   1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                         'kernel': ['rbf', 'poly', 'sigmoid', 'linear']},
             scoring=make_scorer(multi_objective_score, greater_is_better=False),
             verbose=2)

In [49]:
grid_search.best_params_

{'C': 100, 'degree': 1, 'gamma': 2, 'kernel': 'rbf'}

In [50]:
best_grid = grid_search.best_estimator_

In [51]:
y_pred_pmic = best_grid.predict(X_test)

In [52]:
mean_absolute_percentage_error(y_test_pmic, y_pred_pmic)

12.89768027641585

In [53]:
np.std(y_test_pmic), np.std(y_pred_pmic)

(3.8969081681731246, 3.010052083292079)

In [54]:
mean_absolute_percentage_error(y_test_mic, y_pred_mic)

483.09046577157153

In [230]:
mean_squared_error(y_test_mic,y_pred_mic)

4081.405827081923

In [55]:
svr = SVR(C=100, gamma=2)
cv_score = cross_val_score(svr, X_train, y_train_pmic, cv=5, scoring=mape_scorer)
cv_mean = cv_score.mean()
print(cv_score, cv_mean)

_ = svr.fit(X_train, y_train_pmic)
y_pred = svr.predict(X_test)

print(np.std(y_test_pmic), np.std(y_pred))

[-12.31431063 -13.81121271 -15.4761578  -12.77518461 -16.37983947] -14.151341046288062
3.8969081681731246 3.010052083292079


In [56]:
ape = 100*np.abs(y_test_pmic-y_pred)/y_test_pmic

In [57]:
y_pred_mic = np.exp(-y_pred)/1e-6

In [58]:
ape_mic = 100*np.abs(y_test_mic-y_pred_mic)/y_test_mic

In [60]:
ape_mic.mean()

555.4551158730234

In [59]:
list(zip(y_test_pmic.round(4), y_pred.round(4), ape, y_test_mic.round(4), y_pred_mic.round(4), ape_mic))

[(10.2265, 12.6447, 23.647145952663486, 36.2, 3.2246, 91.09238714214806),
 (18.526, 17.7768, 4.044293884025271, 0.009, 0.019, 111.54076867484403),
 (9.2103, 11.7577, 27.658076170129412, 100.0, 7.8285, 92.17152889017031),
 (13.6332, 11.4541, 15.983691962744865, 1.2, 10.6059, 783.8232689373222),
 (12.0577, 14.1074, 16.99947708922244, 5.8, 0.7469, 87.12313503277298),
 (11.4176, 12.0707, 5.719537812137583, 11.0, 5.7251, 47.953613838889346),
 (16.8731, 15.7625, 6.582039285488537, 0.047, 0.1427, 203.61651974949717),
 (11.3306, 12.4106, 9.531395487848714, 12.0, 4.0753, 66.03924761143055),
 (8.0629, 10.1464, 25.84032659289486, 315.0, 39.2159, 87.55049709629547),
 (10.2862, 10.7164, 4.181796228351361, 34.1, 22.1791, 34.958748214350024),
 (12.2061, 11.7659, 3.6062291790529537, 5.0, 7.7649, 55.29851055480915),
 (19.337, 18.1491, 6.143194798808508, 0.004, 0.0131, 228.02112504597136),
 (8.9029, 9.8167, 10.264677287329773, 136.0, 54.533, 59.902227590784626),
 (20.0301, 18.3779, 8.24849379223685, 0.0

### Leave one out CV

In [61]:
loo = LeaveOneOut()

In [62]:
from tqdm import tqdm

In [63]:
result_df = pd.DataFrame(columns = list(df_hiv_cov.columns)+["y_pred_pmic", "y_pred_mic", "ape_pmic", "ape_mic"])
for train_index, test_index in tqdm(loo.split(X)):
    X_train, X_test = X.iloc[train_index,:], X.iloc[test_index,:]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    y_train_pmic, y_train_mic = y_train['pIC50'], y_train['MIC']
    y_test_pmic, y_test_mic = y_test['pIC50'], y_test['MIC']
    svr = SVR(C=100, gamma=2)
    _ = svr.fit(X_train, y_train_pmic)
    y_pred_pmic = svr.predict(X_test)
    y_pred_mic = np.exp(-y_pred_pmic)/1e-6
    ape = 100*np.abs(y_test_pmic-y_pred_pmic)/y_test_pmic
    ape_mic = 100*np.abs(y_test_mic-y_pred_mic)/y_test_mic
    df_val = df_hiv_cov.iloc[test_index,:].values[0].tolist()
    res = np.append(df_val, [y_pred_pmic[0], y_pred_mic[0], ape.values[0], ape_mic.values[0]])
    res = pd.DataFrame([res], columns = list(df_hiv_cov.columns)+["y_pred_pmic", "y_pred_mic", "ape_pmic", "ape_mic"])
    result_df = result_df.append(res)
result_df = result_df[["Sequence", "pIC50", "y_pred_pmic", "ape_pmic", "MIC", "y_pred_mic", "ape_mic"]]

292it [00:07, 38.99it/s]


In [69]:
result_df#.to_csv("../results/SVM_HIV_CoV_pMIC_to_MIC_rbf_c_100_gamma_2.csv", index=False)

,Sequence,pIC50,y_pred_pmic,ape_pmic,MIC,y_pred_mic,ape_mic
0,ACWAAGIKQEF,8.00736806798383,9.15468492794739,14.32826429636626,333.0,105.72333449107087,68.25125090358232
0,ACWGAGIKQEF,8.00736806798383,10.05507815613964,25.572823314357795,333.0,42.967000972162516,87.09699670505631
0,ACWWAGIKAEF,8.00736806798383,7.347887355215854,8.235923554017742,333.0,643.9513647953488,93.37878822683147
0,ACWWAGIKQAF,8.00736806798383,7.183783346973583,10.285336130647186,333.0,758.7916327052899,127.86535516675372
0,ACWWAGIRQEF,8.00736806798383,9.592938827666533,19.801397240903064,333.0,68.20867323503198,79.51691494443483
...,...,...,...,...,...,...,...
0,YTSLIHSLIEEGQNQQEKNEQELLELDKWASLWNWF,20.72326583694641,16.36707431149649,21.02077712907343,0.001,0.07795966093792857,7695.966093792857
0,YTSLIQSLIEESQNQQEKNEQQLLELDKWASLWNWF,20.72326583694641,16.841347179955527,18.732176132538033,0.001,0.048517213717842296,4751.72137178423
0,QKQIANQFNKAISQIQESLTTTSTALGKLQDVVNQNAQALNTLVKQ,12.436744463265175,12.429968233327145,0.05448556057451384,3.97,3.996992984903203,0.679924052977403
0,QNQSANQFQKEISQINEVLTTTNTSLGKLQDDVNQNNQSLNTLQKE,12.192169740361182,12.8373379204455,5.291660088593917,5.07,2.6595917604991226,47.54256882644729


In [68]:
result_df['ape_pmic'].astype('float').mean()

14.024493988093818

In [66]:
result_df['ape_mic'].astype('float').mean()

23955.20035866735